In [63]:
!pip install pyarrow
!pip install beutifulSoup4

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import time
import re
import pyarrow as pa
import pyarrow.parquet as pq

In [94]:
BASIC_URL = "https://www.clien.net/service/search?q={query}&sort=recency&p={page_num}&boardCd=&isBoard=false"
CLIEN_URL = "https://www.clien.net"
REQUEST_REST = 0.25

query = "벤츠 배터리 화재"
queryList = ["벤츠", "배터리", "화재"]
page = 0

posts_url= []

# full_url = BASIC_URL.format(query=urllib.parse.quote(query), page_num=page)

for p in range(16):
    full_url = BASIC_URL.format(query=urllib.parse.quote(query), page_num=p)

    response = requests.get(full_url)
    if response.status_code != 200:
        print("status code is not 200")
        continue
    soup = BeautifulSoup(response.content, "html.parser")
    posts_raw = soup.find_all("a", class_="subject_fixed")

    for link in posts_raw:
        print(link.text)
        posts_url.append((link["href"]))
    
    time.sleep(REQUEST_REST)

에어부산 배터리 화재 이상하네요 ㄷㄷㄷ
다들 블박 보조배터리 사용하시나요?
니로EV 와 아토3 비교 리뷰
모델Y에서 볼보 XC60으로 환승 후 6개월
EQE SUV 500 24년식은 결국 배터리가 변경되었습니다.
요즘 전기차 배터리는 각형 배터리가 대세라네요
EQE 전기차 반년 후기
인천 벤츠 전기차 화재 이유 결국 ‘미스터리’로
중국 자동차 기술 어마 어마하네요.
주차 차량에 불 나 170여 대 피해…보험사 “배상 불가” 이유는?
기아 북미 CEO - 'EV 보조금을 없애는건 멍청한짓'
수정) 벤츠 전기차 유저분들께 사과의 말씀드립니다
2025 올뉴 벤츠 CLA - EV 기술들
노사장이 칭찬한 차량
벤츠 전기차 또 화재가 났지만 양상이 좀 다릅니다.
아파트 지하주차장 전기차? 충전시설? 화재 .news
25년도 전기차 보조금 삭감
오라는 리콜 통지는 안 오고(ev6 - 무상수리 통지문)
타이칸 신규 리콜 (충전제한권장ㅋ)
한국판매 벤츠에 중국산 배터리 그대로 쓰려는 이유.txt
벤츠 eq 시리즈 대란은 거의 품절 난 것 같군요
벤츠가 한국에서 배짱 장사하는이유
벤츠측에서는 결국 파라시스를 계속 쓰겠다고 하는군요
청라 벤츠 EQE 화재는 설계 결함 아닐까요?
테슬라 모델 Y RWD, 1년 운행후기
화난 벤츠 전기차 차주들, 독일 본사 상대로 소송
르노 콜레오스 택갈인거야 알았지만...
과거의 자신과 싸우는 오토기어 김정민씨
청라 벤츠화재 국과수조사결과 뉴스는 좀 아쉽네요
인천 청라 전기차 화재 '외부 충격 배터리 손상 개연성'
청라 벤츠 EQE 국립과학수사연구 감정결과가 뉴스로 나왔네요.
벤츠 배터리 관련 소비자원 답변이 공유되었습니다.....
전기차화재시 사용하는 질식소화포 이거 좋더군요..
청라 전기차 화재 피해 아파트 보니 참혹하네요.
청라 아파트 화재ㅡ 천장마감이 문제라는데
[비굴당] 청라 화재를 보면서 우리나라에 정신병자들이 많구나 싶습니다...
PD수첩 청라 화재 초기 영상
벤츠 전기차 기존 고객에게 주는 혜택 요약
서울시 전기차 충전율 90% 제한

In [ ]:
all_posts = []
all_comments = []

for post_url in posts_url[:10]:
    full_url = CLIEN_URL + post_url
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    }
    
    response = requests.get(full_url, headers=headers)
    if response.status_code != 200:
        print(f"{full_url} - Status Code: {response.status_code}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")

    # 게시글 정보 수집
    post_data = {
        "platform": "clien",
        "keyword": query,
        "post_id": soup.find("input", id="boardSn").get("value"),
        "title": soup.find("h3", class_="post_subject").find_all("span")[0].text,
        "url": full_url,
        "content": soup.find("div", class_="post_article").get_text(separator="\n", strip=True),
        "hits": soup.find("div", class_="post_author").find("span", class_="view_count").find("strong").text,
        "created_at": re.search(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", soup.find("span", class_="view_count date").text).group(0),
        "like_count": soup.find("a", class_="symph_count").find("strong").text if soup.find("a", class_="symph_count") else "0",
        "dislike_count": "0",
        "comment_count": soup.find("a", class_="post_reply").find("span").text if soup.find("a", class_="post_reply") else "0"
    }
    
    all_posts.append(post_data)

    # 댓글 정보 수집
    comments_raw = soup.find_all("div", class_="comment_row")
    
    for row in comments_raw:
        if "blocked" in row.get("class", []):
            continue  # 차단된 댓글 제외

        comment_content = row.find("div", class_="comment_view").get_text(separator="\n", strip=True)
        comment_created_at = row.find("span", class_="timestamp").get_text(strip=True)
        comment_created_at = re.search(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", comment_created_at).group(0)

        comment_like = row.find("div", class_="comment_content_symph")
        comment_like = comment_like.find("strong").text if comment_like else "0"

        comment_data = {
            "post_id": post_data["post_id"],
            "comment_content": comment_content,
            "comment_created_at": comment_created_at,
            "comment_like": comment_like,
            "comment_dislike": None
        }
        all_comments.append(comment_data)

    print(f"{full_url} - Done!")
    time.sleep(REQUEST_REST)

# Parquet 저장
post_table = pa.Table.from_pydict({key: [d[key] for d in all_posts] for key in all_posts[0]})
comments_table = pa.Table.from_pydict({key: [d[key] for d in all_comments] for key in all_comments[0]})

pq.write_table(post_table, "posts.parquet")
pq.write_table(comments_table, "comments.parquet")

print("모든 데이터 Parquet 저장 완료!")

# 이하는 더미 데이터

In [97]:
for post_url in posts_url:
    full_url = CLIEN_URL + post_url
    headers = {
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding":"gzip, deflate, br, zstd",
        "Accept-Language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control":"no-cache",
        "Connection":"keep-alive",
        "Host":"www.clien.net",
        "Pragma":"no-cache",
        "Sec-Ch-Ua":'"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
        "Sec-Ch-Ua-Mobile":"?0",
        "Sec-Ch-Ua-Platform":'"macOS"',
        "Sec-Fetch-Dest":"document",
        "Sec-Fetch-Mode":"navigate",
        "Sec-Fetch-Site":"none",
        "Sec-Fetch-User":"?1",
        "Upgrade-Insecure-Requests":"1",
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
    }
    response = requests.get(full_url,headers=headers)
    if response.status_code != 200:
        print("status code is not 200")
        continue
    soup = BeautifulSoup(response.content, "html.parser")
    
    platform = "clien"
    keyward = query
    post_id = soup.find("input", id="boardSn").get("value")
    title = soup.find("h3", class_="post_subject").find_all("span")[0].text
    url = full_url
    content = soup.find("div", class_="post_article").get_text(separator="\n", strip=True)
    hits = soup.find("div", class_="post_author").find_all("span")[0].find("strong").text
    created_at = soup.find("span", class_="view_count date").get_text(strip=True)
    like_count = soup.find("a", class_="symph_count").find("strong").text
    dislike_count = 0
    comment_count = soup.find("a", class_="post_reply").find("span").text

    comments = []
    comments_raw = soup.find_all("div", class_="comment_row")
    for row in comments_raw:
        if "blocked" in set(row["class"]):
            continue
        comment_content = row.find("div", class_="comment_view").get_text(separator="\n", strip=True)
        commnet_created_at = row.find("span", class_="timestamp").get_text(strip=True)
        commnet_created_at = re.search(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", commnet_created_at).group(0)
        comment_like = row.find("div", class_="comment_content_symph")
        if comment_like:
            comment_like = comment_like.find("strong").text
        else:
            comment_like = 0
        comment_dislike = None

        comment_data = {
            "post_id": post_data["post_id"],
            "comment_content": comment_content,
            "comment_created_at": comment_created_at,
            "comment_like": comment_like,
            "comment_dislike": None
        }
        comments.append(comment_data)


    post_data = {
        "platform": "clien",
        "keyword": query,
        "post_id": soup.find("input", id="boardSn").get("value"),
        "title": soup.find("h3", class_="post_subject").find_all("span")[0].text,
        "url": full_url,
        "content": soup.find("div", class_="post_article").get_text(separator="\n", strip=True),
        "hits": soup.find("div", class_="post_author").find_all("span")[0].find("strong").text,
        "created_at": soup.find("span", class_="view_count date").get_text(strip=True),
        "like_count": soup.find("a", class_="symph_count").find("strong").text,
        "dislike_count": 0,
        "comment_count": soup.find("a", class_="post_reply").find("span").text
    }
    print(full_url, "done!")
    time.sleep(REQUEST_REST)



    

https://www.clien.net/service/board/park/18908101?combine=true&q=%EB%B2%A4%EC%B8%A0+%EB%B0%B0%ED%84%B0%EB%A6%AC+%ED%99%94%EC%9E%AC&p=0&sort=recency&boardCd=&isBoard=false done!
https://www.clien.net/service/board/cm_car/18901334?combine=true&q=%EB%B2%A4%EC%B8%A0+%EB%B0%B0%ED%84%B0%EB%A6%AC+%ED%99%94%EC%9E%AC&p=0&sort=recency&boardCd=&isBoard=false done!
https://www.clien.net/service/board/park/18900530?combine=true&q=%EB%B2%A4%EC%B8%A0+%EB%B0%B0%ED%84%B0%EB%A6%AC+%ED%99%94%EC%9E%AC&p=0&sort=recency&boardCd=&isBoard=false done!
https://www.clien.net/service/board/cm_car/18897141?combine=true&q=%EB%B2%A4%EC%B8%A0+%EB%B0%B0%ED%84%B0%EB%A6%AC+%ED%99%94%EC%9E%AC&p=0&sort=recency&boardCd=&isBoard=false done!
https://www.clien.net/service/board/cm_car/18894996?combine=true&q=%EB%B2%A4%EC%B8%A0+%EB%B0%B0%ED%84%B0%EB%A6%AC+%ED%99%94%EC%9E%AC&p=0&sort=recency&boardCd=&isBoard=false done!
https://www.clien.net/service/board/park/18867084?combine=true&q=%EB%B2%A4%EC%B8%A0+%EB%B0%B0%ED%84%B0%EB%A6%

KeyboardInterrupt: 

In [1]:
all_posts = []
all_comments = []

for post_url in posts_url[:10]:
    full_url = CLIEN_URL + post_url
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    }
    
    response = requests.get(full_url, headers=headers)
    if response.status_code != 200:
        print(f"{full_url} - Status Code: {response.status_code}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")

    # 게시글 정보 수집
    post_data = {
        "platform": "clien",
        "keyword": query,
        "post_id": soup.find("input", id="boardSn").get("value"),
        "title": soup.find("h3", class_="post_subject").find_all("span")[0].text,
        "url": full_url,
        "content": soup.find("div", class_="post_article").get_text(separator="\n", strip=True),
        "hits": soup.find("div", class_="post_author").find("span", class_="view_count").find("strong").text,
        "created_at": re.search(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", soup.find("span", class_="view_count date").text).group(0),
        "like_count": soup.find("a", class_="symph_count").find("strong").text if soup.find("a", class_="symph_count") else "0",
        "dislike_count": "0",
        "comment_count": soup.find("a", class_="post_reply").find("span").text if soup.find("a", class_="post_reply") else "0"
    }
    
    all_posts.append(post_data)

    # 댓글 정보 수집
    comments_raw = soup.find_all("div", class_="comment_row")
    
    for row in comments_raw:
        if "blocked" in row.get("class", []):
            continue  # 차단된 댓글 제외

        comment_content = row.find("div", class_="comment_view").get_text(separator="\n", strip=True)
        comment_created_at = row.find("span", class_="timestamp").get_text(strip=True)
        comment_created_at = re.search(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", comment_created_at).group(0)

        comment_like = row.find("div", class_="comment_content_symph")
        comment_like = comment_like.find("strong").text if comment_like else "0"

        comment_data = {
            "post_id": post_data["post_id"],
            "comment_content": comment_content,
            "comment_created_at": comment_created_at,
            "comment_like": comment_like,
            "comment_dislike": None
        }
        all_comments.append(comment_data)

    print(f"{full_url} - Done!")
    time.sleep(REQUEST_REST)

all_comments.append({"post_id": post_data["post_id"],
            "comment_content": comment_content})

# Parquet 저장
post_table = pa.Table.from_pydict({key: [d[key] for d in all_posts] for key in all_posts[0]})
comments_table = pa.Table.from_pydict({key: [d[key] for d in all_comments] for key in all_comments[0]})

pq.write_table(post_table, "posts.parquet")
pq.write_table(comments_table, "comments.parquet")

print("모든 데이터 Parquet 저장 완료!")

NameError: name 'posts_url' is not defined

In [57]:
post_url = posts_url[0]
full_url = CLIEN_URL + post_url
headers = {
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding":"gzip, deflate, br, zstd",
    "Accept-Language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control":"no-cache",
    "Connection":"keep-alive",
    "Host":"www.clien.net",
    "Pragma":"no-cache",
    "Sec-Ch-Ua":'"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
    "Sec-Ch-Ua-Mobile":"?0",
    "Sec-Ch-Ua-Platform":'"macOS"',
    "Sec-Fetch-Dest":"document",
    "Sec-Fetch-Mode":"navigate",
    "Sec-Fetch-Site":"none",
    "Sec-Fetch-User":"?1",
    "Upgrade-Insecure-Requests":"1",
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
}
response = requests.get(full_url,headers=headers)
if response.status_code != 200:
    print("status code is not 200")

soup = BeautifulSoup(response.content, "html.parser")

'\ntitle = soup.find("h3", class_="post_subject").find_all("span")[0]\ncomment_count = soup.find("a", class_="post_reply").find("span").text\nhits = soup.find("span", class_="view_count").find("strong")\ncreated_at = soup.find("span", class_="view_count date").get_text(strip=True)\ncontent = soup.find("div", class_="post_article").get_text(separator="\n", strip=True)\n\nprint(title, comment_count, hits, created_at, content)\n'

In [2]:
import re
platform = "clien"
keyward = query
post_id = soup.find("input", id="boardSn").get("value")
title = soup.find("h3", class_="post_subject").find_all("span")[0].text
url = CLIEN_URL+post_url
content = soup.find("div", class_="post_article").text
hits = soup.find("div", class_="post_author").find_all("span")[0].find("strong").text
created_at = soup.find("span", class_="view_count date").get_text(strip=True)
like_count = soup.find("a", class_="symph_count").find("strong").text
dislike_count = 0
comment_count = soup.find("a", class_="post_reply").find("span").text

comments_raw = soup.find_all("div", class_="comment_row")
print(platform, keyward, post_id, title, url, comment_count, hits, created_at, content, sep="\n")

for row in comments_raw:
    if "blocked" in set(row["class"]):
        continue
    comment_content = row.find("div", class_="comment_view").text
    commnet_created_at = row.find("span", class_="timestamp").text
    commnet_created_at = re.search(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", commnet_created_at).group(0)
    comment_like = row.find("div", class_="comment_content_symph")
    if comment_like:
        comment_like = comment_like.find("strong").text
    else:
        comment_like = 0
    comment_dislike = None
    print(comment_content, commnet_created_at, comment_like, comment_dislike, sep="\n")


NameError: name 'query' is not defined

In [ ]:
url = 'http://www.clien.net/service/search?q=%EB%B2%A4%EC%B8%A0%20%EB%B0%B0%ED%84%B0%EB%A6%AC%20%ED%99%94%EC%9E%AC&sort=recency&p={p}&boardCd=&isBoard=false'
proxy = {
    'http':'47.251.122.81:18080',
}
sc = 200
page = 0
while sc == 200:
    response = requests.get(url, proxies=proxy)
    url.format(p=page)
    #response = requests.get(url)
    print(response.status_code)
    sc = response.status_code
    page += 1

KeyboardInterrupt: 